In [1]:
import sys, os
sys.path.append('/work')
from train.utils.create_model import create_unet
import yaml
import torchinfo

In [18]:
with open('/work/train/segmentation/train_config/segmentation_culane.yaml', 'r') as f:
    train_config = yaml.load(f, Loader=yaml.Loader)
train_config

{'train': {'loss': {'loss_type': 'lane_dice',
   'lane_dice': {'num_cls': 5},
   'cross_entropy': {'weights': 'None',
    'ignore_index': -100,
    'reduction': 'mean',
    'label_smoothing': 0.0},
   'dice': {'weights': 'None', 'num_cls': 5}},
  'optimizer': {'optimizer_type': 'adamw',
   'adamw': {'lr': 0.0006,
    'betas': [0.9, 0.999],
    'eps': '1e-8',
    'weight_decay': 0.01,
    'amsgrad': False,
    'maximize': False,
    'foreach': 'None',
    'capturable': False,
    'differentialable': False,
    'fused': 'None'},
   'adam': {'lr': 0.0006,
    'betas': [0.9, 0.999],
    'eps': '1e-8',
    'weight_decay': 0.01,
    'amsgrad': False,
    'foreach': 'None',
    'maximize': False,
    'capturaable': False,
    'differentialable': False,
    'fused': 'None'},
   'rmsprop': {'lr': 0.0006,
    'alpha': 0.99,
    'eps': '1e-8',
    'weight_decay': 0,
    'momentum': 0,
    'centered': False,
    'capturable': False,
    'foreach': 'None',
    'maximize': False,
    'differentialab

In [ ]:
# import time

# # dictionary takes approach to its values as depth (level) of the tree
# # if you want to traverse all the values in the dictionary, you need to use recursive function
# def traverse_dict(input_dict):
    
#     start_t = time.time()
    
#     # we will use FIFO style search using queue
#     queue_k = []
#     queue_v = []
#     tree_keys = input_dict.keys()
#     search_flag = True # at first, we need to search what keys they have (search root keys)
#     sub_dict = input_dict # sub_dict is intialized as input_dict
    
#     level_indicator = [] # used to store the level information of the tree (pivot value of current position)
#     key_history = [] # store popped key (not leaf node key)
    
#     while True:
#         if len(level_indicator)==0: # if level indicator is empty, we are at the root level
#             counter = 0
#         else:
#             counter = level_indicator[-1] # cumulative sum
            
#         if search_flag: # if pass then, continue to next iteration
#             for tree_key in tree_keys:
#                 counter += 1
#                 queue_k.insert(0, tree_key)
#                 queue_v.insert(0, sub_dict[tree_key])
#             level_indicator.append(counter)

#         if not isinstance(queue_v[0], dict):
#             key = queue_k.pop(0)
#             leaf = queue_v.pop(0)
            
#             if leaf == 'None': # 1. if leaf element is string 'None' we convert it to python None
#                 leaf = None
#             elif isinstance(leaf, str): # 2. if leaf element is string, we try to convert to float format if possible (convert '1e-8' to float 1e-8)
#                 try:
#                     leaf = float(leaf)
#                 except:
#                     pass
            
#             current_level = level_indicator.pop()
#             if len(level_indicator) > 1: # in this case, leaf belongs to the sub-dictionary
#                 if (current_level != level_indicator[-1]):
#                     current_level -= 1 # decrease the pivot by 1
#                     level_indicator.append(current_level)
#                     sub_dict[key] = leaf # update the leaf value to filtered value
#                 else: # in this case, leaf does not belong to the sub-dictionary, we need to move to the parent node (directly corresponding value is not dictionary)
#                     access = input_dict
#                     for k in key_history:
#                         print("key: ", k)
#                         access = access[k]
#                     access[key] = leaf # follow the key history and save the leaf value
#                     level_indicator.append(current_level)
                    
#                 current_level = level_indicator.pop()
#                 if (current_level == level_indicator[-1]):
#                     key_history.pop() # remove the registered key and level_indicator
#                 else:
#                     level_indicator.append(current_level)
                    
#             elif len(level_indicator) == 1: # in this case, leaf directly belongs to the root
#                 current_level -= 1
#                 level_indicator.append(current_level)
#                 sub_dict[key] = leaf
                
#             print("leaf value: ", leaf) # no more depth search, left node
#             search_flag=False
#         else:
#             key = queue_k.pop(0) # in case of dictionary, we need to pop the key on the stored queue
#             sub_dict = queue_v.pop(0)
#             tree_keys = sub_dict.keys()
#             search_flag=True # more depth search is needed
                        
#             key_history.append(key) # since this is not the leaf node key, we need to store it to save the information
#         if len(queue_v)==0 and not search_flag: # consuming all the element in queue means we search all the trees
#             break
    
#     end_t = time.time()
#     print("Elasped time: ", end_t-start_t)
            
# traverse_dict(train_config)

leaf value:  False
leaf value:  10000
leaf value:  60
leaf value:  cos
leaf value:  0.3
leaf value:  0.0006
leaf value:  60
leaf value:  cosine_warmup_restart
leaf value:  30
leaf value:  1e-06
leaf value:  10
leaf value:  100
leaf value:  1.0
leaf value:  0.2
leaf value:  1e-05
leaf value:  60
leaf value:  cosine_warmup_restart
leaf value:  None
leaf value:  False
leaf value:  None
leaf value:  False
leaf value:  False
leaf value:  0
leaf value:  0
leaf value:  0
leaf value:  0.0006
leaf value:  False
leaf value:  False
leaf value:  None
leaf value:  False
leaf value:  False
leaf value:  0
leaf value:  0
leaf value:  1e-08
leaf value:  0.99
leaf value:  0.0006
leaf value:  None
leaf value:  False
leaf value:  False
leaf value:  False
leaf value:  None
leaf value:  False
leaf value:  0.01
leaf value:  1e-08
leaf value:  [0.9, 0.999]
leaf value:  0.0006
leaf value:  None
leaf value:  False
leaf value:  False
leaf value:  None
leaf value:  False
leaf value:  False
leaf value:  0.01
leaf 

In [ ]:
with open('/work/train/segmentation/model_config/unet.yaml') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
    
unet = create_unet('/work/train/segmentation/model_config/unet.yaml')
torchinfo.summary(unet, (1, 3, 224, 672)) # torchsummary takes some gpu memory (if only loaded with cpu device, no gpu memory is used)

Layer (type:depth-idx)                                  Output Shape              Param #
UNet                                                    [1, 5, 224, 672]          --
├─Encoder: 1-1                                          [1, 384, 7, 21]           --
│    └─Stem: 2-1                                        [1, 48, 56, 168]          --
│    │    └─Conv2d: 3-1                                 [1, 48, 56, 168]          2,352
│    └─ModuleList: 2-2                                  --                        --
│    │    └─Stage: 3-2                                  [1, 48, 56, 168]          2,780,793
│    │    └─Stage: 3-3                                  [1, 96, 28, 84]           1,634,994
│    │    └─Stage: 3-4                                  [1, 192, 14, 42]          3,489,480
│    │    └─Stage: 3-5                                  [1, 384, 7, 21]           4,653,000
├─Decoder: 1-2                                          [1, 48, 224, 672]         --
│    └─ModuleList: 2-3       